In [19]:
import streamlit as st
import pickle
import pandas as pd
import numpy as np

# Load model & encoders
with open('models/ensemble_model.pkl', 'rb') as ensemble:
    model = pickle.load(ensemble)

with open('models/dummy_info.pkl', 'rb') as dummy:
    dummy_info = pickle.load(dummy)

with open('models/preprocessor.pkl', 'rb') as process:
    preprocess = pickle.load(process)

with open('models/selected_features.pkl', 'rb') as features:
    selected_features = pickle.load(features)

In [20]:
import pandas as pd
pd.set_option('display.max_columns', None)

In [21]:
df = pd.DataFrame({
    "body_type": ["overweight"],
    "sex": ["female"],
    "diet": ["pescatarian"],
    "shower_frequency": ["daily"],
    "heating_energy_source": ["coal"],
    "transport": ["public"],
    "social_activity": ["often"],
    "monthly_grocery_bill": [230],
    "air_travel_frequency": ["frequently"],
    "vehicle_monthly_distance_km": [210],
    "waste_bag_size": ["large"],
    "waste_bag_weekly_count": [4],
    "tv_pc_daily_hours": [7],
    "new_clothes_monthly": [2],
    "internet_daily_hours": [8],
    "energy_efficiency": ["No"],
    "recycling": [["Metal", "Paper"]],
    "cooking_with": [["Stove", "Oven"]]
})


In [22]:
df.head()

,body_type,sex,diet,shower_frequency,heating_energy_source,transport,social_activity,monthly_grocery_bill,air_travel_frequency,vehicle_monthly_distance_km,waste_bag_size,waste_bag_weekly_count,tv_pc_daily_hours,new_clothes_monthly,internet_daily_hours,energy_efficiency,recycling,cooking_with
0,overweight,female,pescatarian,daily,coal,public,often,230,frequently,210,large,4,7,2,8,No,"[Metal, Paper]","[Stove, Oven]"


In [25]:
def create_dummy_variables(X, column_name):
    X = X.copy()

    unique_values = set()

    # Iterate over the column directly (no need for ast.literal_eval)
    for item in X[column_name].dropna():
        unique_values.update(item)  # Update the set with list elements

    # Create dummy variables
    for value in unique_values:
        X[value] = X[column_name].apply(lambda x: int(value in x) if pd.notnull(x) else 0)

    X.drop(columns=column_name, inplace=True)

    return X, unique_values

In [26]:

X, recycling_categories = create_dummy_variables(df, 'recycling')
X, cooking_categories = create_dummy_variables(df, 'cooking_with')


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()